# Objetivo:
O objetivo deste projeto é desenvolver um modelo de previsão utilizando o PySpark para determinar se um cliente é provável de possuir um alto limite de cartão de crédito ou não. O foco será em identificar os clientes cujos limites estão acima de 4549 dólares, considerando que a metade dos dados está abaixo desse valor. Serão utilizadas duas categorias para representar os clientes em relação aos seus limites: limite igual a 0 indicará que o cliente possui um limite de até 4549 dólares, enquanto limite igual a 1 indicará que o cliente possui um limite acima desse valornstituição financeira.édito mais elevados.dito mais elevados.

## Subobjetivo:

- Pré-processamento dos Dados: Realizar a limpeza e transformação dos dados brutos, incluindo a identificação e tratamento de valores ausentes e inconsistentes.

- Análise Exploratória: Realizar uma análise exploratória dos dados para entender a distribuição dos limites de cartão, identificar padrões e tendências, e explorar relações com outras variáveis relevantes.

- Engenharia de Recursos: Criar e selecionar recursos (features) relevantes para o modelo preditivo, considerando informações que possam impactar a decisão sobre os limites de cartão dos clientes.

- Modelagem: Desenvolver e treinar um modelo de previsão utilizando algoritmos do PySpark, como por exemplo, algoritmos de classificação como Decision Trees, Random Forest ou Gradient Boosting.

- Avaliação do Modelo: Avaliar o desempenho do modelo utilizando métricas apropriadas, como acurácia, precisão, recall, F1-score, entre outras. Realizar ajustes e otimizações no modelo, se necessário.

- Interpretação dos Resultados: Interpretar os resultados do modelo, identificando quais variáveis são mais relevantes para a previsão e como elas influenciam na decisão de atribuir limites de cartão.

## Resumo do Projeto:
Este projeto visa desenvolver um modelo de previsão utilizando o PySpark para determinar se um cliente possui um alto limite de cartão de crédito (acima de 4549 dólares) ou não. Através de uma abordagem de aprendizado de máquina, serão explorados os dados disponíveis, realizada a engenharia de recursos e desenvolvido um modelo preditivo. O modelo passará por avaliação rigorosa e otimização para garantir sua eficácia na identificação dos clientes com limites elevados. O resultado final será um modelo pronto para implantação, capaz de auxiliar em decisões relacionadas aos limites de cartão de crédito dos clientes, contribuindo para uma abordagem mais precisa e informada por parte da instituição financeira.

# Projeto:

In [2]:
# Instalando o pyspark
!pip install pyspark

^C


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
dados = spark.read.csv('cartaorenda.csv', header=True, sep=',', inferSchema=True )
dados.toPandas()

,_c0,Idade,Sexo,Dependentes,Nivel_educacional,Estado_civil,Renda,limite_do_cartao2
0,0,51,M,3,Graduate,Married,$80K - $120K,0
1,1,51,M,3,Graduate,Married,$80K - $120K,0
2,2,40,F,4,High School,Unknown,Less than $40K,0
3,3,44,M,2,Graduate,Married,$40K - $60K,0
4,4,44,M,2,Graduate,Married,$40K - $60K,0
...,...,...,...,...,...,...,...,...
10122,10122,55,F,3,Uneducated,Single,Unknown,1
10123,10123,54,M,1,High School,Single,$60K - $80K,1
10124,10124,44,F,1,High School,Married,Less than $40K,1
10125,10125,30,M,2,Graduate,Unknown,$40K - $60K,1


In [4]:
# Agrupando os dados pela coluna limite_do_cartao2
dados.groupBy('limite_do_cartao2').count().show()
# demonstra que os dados estão bem dividos 

+-----------------+-----+
|limite_do_cartao2|count|
+-----------------+-----+
|                1| 5061|
|                0| 5066|
+-----------------+-----+



In [5]:
dados.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Dependentes: integer (nullable = true)
 |-- Nivel_educacional: string (nullable = true)
 |-- Estado_civil: string (nullable = true)
 |-- Renda: string (nullable = true)
 |-- limite_do_cartao2: integer (nullable = true)



In [6]:
colunasBinarias = ['Sexo']
# transformando a coluna binária de sexo 
# M masculino = 1 
# F feminino = 0

In [7]:
from pyspark.sql import functions as f

In [8]:
# trocando o valor de M para 1
todasColunas = [f.when(f.col(c)=='M', 1).otherwise(0).alias(c) for c in colunasBinarias] 
# pegando o nomde da coluna e percorrendo 

In [9]:
todasColunas

[Column<'CASE WHEN (Sexo = M) THEN 1 ELSE 0 END AS Sexo'>]

In [10]:
# reversed colocando os dados binários depois das colunas nativas
for coluna in reversed(dados.columns): 
    if coluna not in colunasBinarias:
        todasColunas.insert(0, coluna)
todasColunas

['_c0',
 'Idade',
 'Dependentes',
 'Nivel_educacional',
 'Estado_civil',
 'Renda',
 'limite_do_cartao2',
 Column<'CASE WHEN (Sexo = M) THEN 1 ELSE 0 END AS Sexo'>]

In [11]:
dados.select(todasColunas).toPandas()

,_c0,Idade,Dependentes,Nivel_educacional,Estado_civil,Renda,limite_do_cartao2,Sexo
0,0,51,3,Graduate,Married,$80K - $120K,0,1
1,1,51,3,Graduate,Married,$80K - $120K,0,1
2,2,40,4,High School,Unknown,Less than $40K,0,0
3,3,44,2,Graduate,Married,$40K - $60K,0,1
4,4,44,2,Graduate,Married,$40K - $60K,0,1
...,...,...,...,...,...,...,...,...
10122,10122,55,3,Uneducated,Single,Unknown,1,0
10123,10123,54,1,High School,Single,$60K - $80K,1,1
10124,10124,44,1,High School,Married,Less than $40K,1,0
10125,10125,30,2,Graduate,Unknown,$40K - $60K,1,1


In [12]:
dataset = dados.select(todasColunas)

In [13]:
dataset.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Idade: integer (nullable = true)
 |-- Dependentes: integer (nullable = true)
 |-- Nivel_educacional: string (nullable = true)
 |-- Estado_civil: string (nullable = true)
 |-- Renda: string (nullable = true)
 |-- limite_do_cartao2: integer (nullable = true)
 |-- Sexo: integer (nullable = false)



In [14]:
dados.select(['Nivel_educacional','Estado_civil','Renda']).toPandas()

,Nivel_educacional,Estado_civil,Renda
0,Graduate,Married,$80K - $120K
1,Graduate,Married,$80K - $120K
2,High School,Unknown,Less than $40K
3,Graduate,Married,$40K - $60K
4,Graduate,Married,$40K - $60K
...,...,...,...
10122,Uneducated,Single,Unknown
10123,High School,Single,$60K - $80K
10124,High School,Married,Less than $40K
10125,Graduate,Unknown,$40K - $60K


# Dummies

In [15]:
# Criando os dummies dessas colunas para que representem apenas 0 e 1 
# pivot recebe o nome da coluna
dataset.groupBy('_c0').pivot('Nivel_educacional').agg(f.lit(1)).na.fill(0).toPandas()

,_c0,College,Doctorate,Graduate,High School,Post-Graduate,Uneducated,Unknown
0,7833,0,0,0,0,0,1,0
1,1342,0,0,1,0,0,0,0
2,7340,0,0,0,0,0,0,1
3,1088,0,0,0,1,0,0,0
4,1238,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
10122,5431,1,0,0,0,0,0,0
10123,401,0,0,0,1,0,0,0
10124,6153,0,0,0,0,0,0,1
10125,3245,0,0,0,0,0,1,0


In [16]:
Nivel_educacional = dataset.groupBy('_c0').pivot('Nivel_educacional').agg(f.lit(1)).na.fill(0)
Estado_civil = dataset.groupBy('_c0').pivot('Estado_civil').agg(f.lit(1)).na.fill(0)
Renda = dataset.groupBy('_c0').pivot('Renda').agg(f.lit(1)).na.fill(0)

In [17]:
Estado_civil.toPandas()

,_c0,Divorced,Married,Single,Unknown
0,7253,0,1,0,0
1,2142,0,0,1,0
2,2659,0,1,0,0
3,1829,0,0,1,0
4,4519,0,0,0,1
...,...,...,...,...,...
10122,3406,0,0,1,0
10123,6634,0,1,0,0
10124,5125,0,1,0,0
10125,1138,0,0,1,0


In [18]:
Renda.toPandas()

,_c0,$120K +,$40K - $60K,$60K - $80K,$80K - $120K,Less than $40K,Unknown
0,6336,0,0,1,0,0,0
1,6654,0,1,0,0,0,0
2,9427,0,1,0,0,0,0
3,3794,0,0,0,0,1,0
4,5803,0,0,0,0,1,0
...,...,...,...,...,...,...,...
10122,2376,0,0,0,0,1,0
10123,9901,0,0,0,1,0,0
10124,4173,0,1,0,0,0,0
10125,5431,0,0,0,0,1,0


In [19]:
Renda.columns

['_c0',
 '$120K +',
 '$40K - $60K',
 '$60K - $80K',
 '$80K - $120K',
 'Less than $40K',
 'Unknown']

In [20]:
# unindo as informações, juntando pelo '_c0', pela linha, e apagando as desnecessárias
dataset\
    .join(Nivel_educacional, '_c0', how='inner')\
    .join(Estado_civil, '_c0', how='inner')\
    .join(Renda, '_c0', how='inner')\
    .select(
    '*',
    f.col('$120K +').alias('+120k'),
    f.col('$40K - $60K').alias('entre40e60'),
    f.col('$60K - $80K').alias('entre60e80'),
    f.col('$80K - $120K').alias('entre80e120'),
    f.col('Less than $40K').alias('menor40'))\
    .drop(
    'Unknown','$120K +','$40K - $60K','$60K - $80K','Less than $40K','$80K - $120K',
    'Nivel_educacional','Renda','Estado_civil','Dependentes','Idade','_c0')\
    .toPandas()    

,limite_do_cartao2,Sexo,College,Doctorate,Graduate,High School,Post-Graduate,Uneducated,Divorced,Married,Single,+120k,entre40e60,entre60e80,entre80e120,menor40
0,1,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
2,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
4,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1
10123,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
10124,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0
10125,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0


In [21]:
dataset = dataset\
    .join(Nivel_educacional, '_c0', how='inner')\
    .join(Estado_civil, '_c0', how='inner')\
    .join(Renda, '_c0', how='inner')\
    .select(
    '*',
    f.col('$120K +').alias('+120k'),
    f.col('$40K - $60K').alias('entre40e60'),
    f.col('$60K - $80K').alias('entre60e80'),
    f.col('$80K - $120K').alias('entre80e120'),
    f.col('Less than $40K').alias('menor40'))\
    .drop(
    'Unknown','$120K +','$40K - $60K','$60K - $80K','Less than $40K','$80K - $120K',
    'Nivel_educacional','Renda','Estado_civil','Dependentes','Idade','_c0')

# Regressão Logística

In [22]:
from pyspark.ml.feature import VectorAssembler
# mudando o nome da coluna de resposta no caso churn para label
dataset = dataset.withColumnRenamed('limite_do_cartao2', 'label')

In [23]:
# salvando as colunas nas variáveis
X = dataset.columns
X.remove('label')# removendo a coluna que queremos prever
X

['Sexo',
 'College',
 'Doctorate',
 'Graduate',
 'High School',
 'Post-Graduate',
 'Uneducated',
 'Divorced',
 'Married',
 'Single',
 '+120k',
 'entre40e60',
 'entre60e80',
 'entre80e120',
 'menor40']

In [24]:
assembler = VectorAssembler(inputCols=X, outputCol='features') 
# output criando o nome do vetor da coluna única com todas variáveis explicativas
dataset_prep = assembler.transform(dataset).select('features', 'label')
dataset_prep.show(10, truncate=False)# truncate mostra tudo

+---------------------------------+-----+
|features                         |label|
+---------------------------------+-----+
|(15,[0,6,7,10],[1.0,1.0,1.0,1.0])|1    |
|(15,[3,9,14],[1.0,1.0,1.0])      |0    |
|(15,[0,9,14],[1.0,1.0,1.0])      |1    |
|(15,[4,9,11],[1.0,1.0,1.0])      |0    |
|(15,[0,1,8,11],[1.0,1.0,1.0,1.0])|0    |
|(15,[4,14],[1.0,1.0])            |0    |
|(15,[0,2,9,12],[1.0,1.0,1.0,1.0])|0    |
|(15,[0,8,11],[1.0,1.0,1.0])      |1    |
|(15,[1,9],[1.0,1.0])             |1    |
|(15,[3,9,14],[1.0,1.0,1.0])      |1    |
+---------------------------------+-----+
only showing top 10 rows



# Ajuste e Previsão

In [25]:
SEED=10
treino, teste = dataset_prep.randomSplit([0.7,0.3], seed=SEED)

In [26]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
modelo_lr = lr.fit(treino)
previsoes_lr_teste = modelo_lr.transform(teste)
previsoes_lr_teste.toPandas()

,features,label,rawPrediction,probability,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[-0.677302171912212, 0.677302171912212]","[0.33686369560760543, 0.6631363043923946]",1.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[-0.677302171912212, 0.677302171912212]","[0.33686369560760543, 0.6631363043923946]",1.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[-0.677302171912212, 0.677302171912212]","[0.33686369560760543, 0.6631363043923946]",1.0
3,"(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0,"[-1.1779729977017899, 1.1779729977017899]","[0.23541685235751889, 0.7645831476424811]",1.0
4,"(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[-1.918218999311589, 1.918218999311589]","[0.12806030268995292, 0.8719396973100471]",1.0
...,...,...,...,...,...
3026,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1.054266513242903, -1.054266513242903]","[0.7415933448261187, 0.2584066551738813]",0.0
3027,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[1.054266513242903, -1.054266513242903]","[0.7415933448261187, 0.2584066551738813]",0.0
3028,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1.054266513242903, -1.054266513242903]","[0.7415933448261187, 0.2584066551738813]",0.0
3029,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1.054266513242903, -1.054266513242903]","[0.7415933448261187, 0.2584066551738813]",0.0


# Métricas

In [27]:
resumo_lr_treino = modelo_lr.summary

In [28]:
print("Acurácia: %f" % resumo_lr_treino.accuracy)# 
print("Precisão: %f" % resumo_lr_treino.precisionByLabel[1])
print("Recall: %f" % resumo_lr_treino.recallByLabel[1])
print("F1: %f" % resumo_lr_treino.fMeasureByLabel()[1])

Acurácia: 0.729284
Precisão: 0.726963
Recall: 0.727995
F1: 0.727479


In [29]:
# matriz de confusão
previsoes_lr_teste.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       1.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    0|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 20 rows



In [30]:
previsoes_lr_teste.select('label','prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).toPandas()

,label,prediction
0,1,1.0
1,1,1.0
2,1,1.0
3,1,1.0
4,1,1.0
...,...,...
1112,1,1.0
1113,1,1.0
1114,1,1.0
1115,1,1.0


In [31]:
previsoes_lr_teste.select('label','prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
# 1103 cliente que preveu ser de alto limite de fato era mesmo

1117

In [32]:
# verdadeiros positivos, true positive
tp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
# verdadeiros negativos, true negative
tn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
# falsos positivos
fp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
# falsos negativos
fn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()
print(tp, tn, fp, fn)

1117 1074 418 422


In [33]:
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn

  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'limite-alto', ' '*5 ,'limite-baixo')
  print(' '*4, 'limite-alto', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'limite-baixo', ' '*5, int(fp/valorN), ' '*10, int(tn/valorN))

In [34]:
calcula_mostra_matriz_confusao(previsoes_lr_teste)

                     Previsto
                limite-alto       limite-baixo
     limite-alto        1117         422
Real
     limite-baixo    418         1074


# Árvore de Decisão - Classificação

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier
dtc = DecisionTreeClassifier(seed=SEED)
modelo_dtc = dtc.fit(treino)
previsoes_dtc_treino = modelo_dtc.transform(treino) # no sklearn é o método predict
previsoes_dtc_treino.toPandas()

,features,label,rawPrediction,probability,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[882.0, 2448.0]","[0.2648648648648649, 0.7351351351351352]",1.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[882.0, 2448.0]","[0.2648648648648649, 0.7351351351351352]",1.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[882.0, 2448.0]","[0.2648648648648649, 0.7351351351351352]",1.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[882.0, 2448.0]","[0.2648648648648649, 0.7351351351351352]",1.0
4,"(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[882.0, 2448.0]","[0.2648648648648649, 0.7351351351351352]",1.0
...,...,...,...,...,...
7091,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1789.0, 516.0]","[0.7761388286334057, 0.22386117136659436]",0.0
7092,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1789.0, 516.0]","[0.7761388286334057, 0.22386117136659436]",0.0
7093,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1789.0, 516.0]","[0.7761388286334057, 0.22386117136659436]",0.0
7094,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[1789.0, 516.0]","[0.7761388286334057, 0.22386117136659436]",0.0


# Métricas

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator # usa 2 ou mais variáveis dependentes
evaluator = MulticlassClassificationEvaluator()
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})

0.730129650507328

In [37]:
# com os dados de treino
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.730130
Precisão: 0.725386
Recall: 0.734242
F1: 0.729787


In [38]:
# agora com os dados de teste 
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [39]:
# com os dados de teste
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.722864
Precisão: 0.725629
Recall: 0.730344
F1: 0.727979


In [40]:
print('Decision Tree Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        2586         936
Real
     limite-baixo    979         2595
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.730130
Precisão: 0.725386
Recall: 0.734242
F1: 0.729787

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        1124         415
Real
     limite-baixo    425         1067
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.722864
Precisão: 0.725629
Recall: 0.730344
F1: 0.727979


# Random Forest - Classificação

In [41]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(seed=SEED)
modelo_rfc = rfc.fit(treino)
previsoes_rfc_treino = modelo_rfc.transform(treino)
previsoes_rfc_treino.toPandas()

,features,label,rawPrediction,probability,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[8.705467879945283, 11.294532120054718]","[0.43527339399726417, 0.5647266060027359]",1.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[8.705467879945283, 11.294532120054718]","[0.43527339399726417, 0.5647266060027359]",1.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[8.705467879945283, 11.294532120054718]","[0.43527339399726417, 0.5647266060027359]",1.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[8.705467879945283, 11.294532120054718]","[0.43527339399726417, 0.5647266060027359]",1.0
4,"(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",1,"[4.517527034425398, 15.482472965574605]","[0.22587635172126985, 0.7741236482787301]",1.0
...,...,...,...,...,...
7091,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[14.781285587394358, 5.218714412605641]","[0.7390642793697179, 0.26093572063028203]",0.0
7092,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[14.781285587394358, 5.218714412605641]","[0.7390642793697179, 0.26093572063028203]",0.0
7093,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[14.781285587394358, 5.218714412605641]","[0.7390642793697179, 0.26093572063028203]",0.0
7094,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[14.781285587394358, 5.218714412605641]","[0.7390642793697179, 0.26093572063028203]",0.0


# Métricas

In [42]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [43]:
print('Random Forest Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        2579         943
Real
     limite-baixo    974         2600
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.729848
Precisão: 0.725865
Recall: 0.732254
F1: 0.729046

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        1123         416
Real
     limite-baixo    424         1068
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.722864
Precisão: 0.725921
Recall: 0.729695
F1: 0.727803


# Otimização dos hiperparâmetros

# Árvore de decisão com Cross Validation

In [ ]:
# cross validation bastante demorado e os resultados não foram melhores, foram pouco piores
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
dtc = DecisionTreeClassifier(seed=SEED)

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(dtc.maxDepth, [2,5,10])\
    .addGrid(dtc.maxBins, [10,15,20])\
    .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
# numFolds é a quantidade da quebra dos dados para realizar o treino, no caso quebrando em 3 grupos
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED)

In [ ]:
import time
inicial = time.time()
modelo_dtc_cv = dtc_cv.fit(treino)
final = time.time()
print('{:.0f} segundos'.format(final -inicial))
# gastou 1447 segundos ou seja 24,11 minutos para rodar

In [ ]:
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [ ]:
print('Decision Tree Classifier - Tuning - CV')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

In [47]:
# retorno quando processou os dados, muito demorado para resultado abaixo dos outros modelos
print('Decision Tree Classifier - Tuning - CV')
print('========================================')
print('Dados de Teste')
print('========================================')
print('Matriz de Confusão')
print('----------------------------------------')
print('                     Previsto')
print('                limite-alto       limite-baixo')
print('     limite-alto        1124         415')
print('Real')
print('     limite-baixo       425            1067')
print('----------------------------------------')
print('Métricas')
print('----------------------------------------')
print('Acurácia: 0.722864')
print('Precisão: 0.725629')
print('Recall: 0.730344')
print('F1: 0.727979')

Decision Tree Classifier - Tuning - CV
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        1124         415
Real
     limite-baixo       425            1067
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.722864
Precisão: 0.725629
Recall: 0.730344
F1: 0.727979


# Random Forest com Cross Validation

In [ ]:
rfc = RandomForestClassifier(seed=SEED)

In [ ]:
grid = ParamGridBuilder()\
    .addGrid(rfc.maxDepth, [2,5])\
    .addGrid(rfc.maxBins, [10,22])\
    .addGrid(rfc.numTrees, [10,15])\
    .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED)

In [ ]:
import time
inicial = time.time()
modelo_rfc_cv = rfc_cv.fit(treino)
final = time.time()
print('{:.1f} minutos'.format((final -inicial)/60))
# foi mais de 50 min resolvi parar com o treinamento
# o intuito é aprender

In [ ]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [ ]:
print('Random Forest Classifier - Tuning - CV')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

In [48]:
# retorno quando processou os dados, muito demorado para resultado abaixo dos outros modelos
print('Random Forest Classifier - Tuning - CV')
print('========================================')
print('Dados de Teste')
print('========================================')
print('Matriz de Confusão')
print('----------------------------------------')
print('                     Previsto')
print('                limite-alto       limite-baixo')
print('     limite-alto        1186         353')
print('Real')
print('     limite-baixo       500            992')
print('----------------------------------------')
print('Métricas')
print('----------------------------------------')
print('Acurácia: 0.718575')
print('Precisão: 0.703440')
print('Recall: 0.770630')
print('F1: 0.735504')

Random Forest Classifier - Tuning - CV
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                limite-alto       limite-baixo
     limite-alto        1186         353
Real
     limite-baixo       500            992
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.718575
Precisão: 0.703440
Recall: 0.770630
F1: 0.735504


# Modelo Final ou Melhor modelo

In [51]:
# O modelo de regressão Linear foi mínimamente inferior a árvore de decisão, mas ela custa menos em processamento
# neste caso escolheria o modelo de regressão linear apesar de todos estarem muito próximos
# os modelos com cross validation se mostraram pouco piores mas o consumo de processamento é muito alto
# mesmo assim irei utilizar o modelo de random Forest com cross validation para demonstrar a retirada dos hiperparâmetros
# mas na realidade utilizaria a regressão linear
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

NameError: name 'modelo_rfc_cv' is not defined

In [50]:
# mostrando os melhores hiperparâmetros
print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees) 
# parâmetro para descobrir o número de árvores

5
10


In [ ]:
# criando o novo modelo com os hiperparâmetros otimizados
rfc_tunning = RandomForestClassifier(maxDepth=5,maxBins=10, numTrees=10, seed=SEED)

In [ ]:
# colocando agora todos os dados sem separação de treino e teste
# o melhor modelo que conseguimos na análise
modelo_rfc_tunning = rfc_tunning.fit(dataset_prep) 

In [ ]:
X

In [ ]:
novo_cliente = [{
    'Sexo': 1,
    'College': 0,
    'Doctorate': 0,
    'Graduate': 0,
    'High School': 0,
    'Post-Graduate': 1,
    'Uneducated': 0,
    'Divorced': 0,
    'Married': 0,
    'Single': 1,
    '+120k': 0,
    'entre40e60': 0,
    'entre60e80': 0,
    'entre80e120': 0,
    'menor40': 1
}]

In [ ]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.toPandas()

In [ ]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [ ]:
novo_cliente_prep = assembler.transform(novo_cliente).select('features')

In [ ]:
novo_cliente_prep.toPandas()

In [ ]:
# de acordo com o modelo seremos capazes saber se este cliente pode estar no alto limite ou baixo limite de cartão de crédito
# de acordo com o modelo rfc com cv
modelo_rfc_tunning.transform(novo_cliente_prep).toPandas()

In [53]:
#resultado
print('	features	rawPrediction	probability	prediction')
print('0	(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...	[7.036260292040566, 2.963739707959435]	[0.7036260292040565, 0.29637397079594346]	0.0')

	features	rawPrediction	probability	prediction
0	(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...	[7.036260292040566, 2.963739707959435]	[0.7036260292040565, 0.29637397079594346]	0.0


In [ ]:
# de acordo com a previsão do random forest com cross validation esse cliente deve começar com baixo limite

# Prevendo com o modelo de Regressão Logística

In [ ]:
lr = LogisticRegression()
modelo_lr_com_todos_dados = lr.fit(dataset_prep) 
novo_cliente = [{
    'Sexo': 1,
    'College': 0,
    'Doctorate': 0,
    'Graduate': 0,
    'High School': 0,
    'Post-Graduate': 1,
    'Uneducated': 0,
    'Divorced': 0,
    'Married': 0,
    'Single': 1,
    '+120k': 0,
    'entre40e60': 0,
    'entre60e80': 0,
    'entre80e120': 0,
    'menor40': 1
}]
novo_cliente = spark.createDataFrame(novo_cliente)
assembler = VectorAssembler(inputCols=X, outputCol='features')
novo_cliente_prep = assembler.transform(novo_cliente).select('features')
modelo_lr_com_todos_dados.transform(novo_cliente_prep).toPandas()

In [55]:
print('features	rawPrediction	probability	prediction')
print('0	(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...	[0.5555909771304192, -0.5555909771304192]	[0.6354317649973102, 0.36456823500268976]	0.0')

features	rawPrediction	probability	prediction
0	(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...	[0.5555909771304192, -0.5555909771304192]	[0.6354317649973102, 0.36456823500268976]	0.0
